In [1]:
import csv
import numpy as np

In [ ]:
database_table_definitions = [['Table_Name', 'Column_Name', 'Data_Type', 'Length', 'PK', 'Nullability', 'FK'],
                              ['Car', 'ID', 'VARCHAR', '10', 'Yes', 'No', 'No'],
                              ['Car', 'Body_Style', 'VARCHAR', '20', 'No', 'Yes', 'No'],
                              ['Car', 'Make', 'VARCHAR', '20', 'No', 'Yes', 'No'],
                              ['Car', 'Model', 'VARCHAR', '20', 'No', 'Yes', 'No'],
                              ['Car', 'Price', 'DECIMAL', '9,2', 'No', 'Yes', 'No'],
                              ['Car', 'Trim_Level', 'VARCHAR', '20', 'No', 'Yes', 'No'],
                              ['Car', 'Year', 'DATE', '0', 'No', 'Yes', 'No'],
                              
                              ['Boat', 'ID', 'VARCHAR', '10', 'Yes', 'No', 'No'],
                              ['Boat', 'Flying_Boat', 'BOOLEAN', '0', 'No', 'Yes', 'No'],
                              ['Boat', 'Name', 'VARCHAR', '20', 'No', 'No', 'No'],
                              
                              ['Flying_Boat', 'Boat_ID', 'VARCHAR', '10', 'Yes', 'No', 'Yes'],
                              ['Flying_Boat', 'Plane_ID', 'VARCHAR', '10', 'Yes', 'No', 'Yes'],
                              ['Flying_Boat', 'Information', 'VARCHAR', '100', 'No', 'Yes', 'No'],
                              
                              ['Plane', 'ID', 'VARCHAR', '10', 'Yes', 'No', 'No'],
                              ['Plane', 'Flying_Boat', 'BOOLEAN', '0', 'No', 'Yes', 'No'],
                              ['Plane', 'Name', 'VARCHAR', '20', 'No', 'No', 'No'],
                              
                              ['Vehicle', 'ID', 'VARCHAR', '10', 'Yes', 'No', 'No'],
                              ['Vehicle', 'Kind', 'VARCHAR', '20', 'No', 'No', 'No'],
                              ['Vehicle', 'Details', 'VARCHAR', '10', 'No', 'Yes', 'Yes']
                             ]

In [ ]:
np.savetxt('vehicle_database_sql_table_definitions.csv', 
           database_table_definitions, 
           delimiter = ',', 
           fmt = '% s')

In [2]:
def read_database_table_definitions_from_csv_file(csv_file_name):
    
    database_table_definitions = []
    
    with open(csv_file_name) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter =',')
        
        for row in csv_reader:
            database_table_definitions.append(row)
    
    return database_table_definitions

In [3]:
def print_database_table_definitions(database_table_definitions):
    
    for row in database_table_definitions:
        print(row)

In [4]:
def determine_table_definitions_column_indexes(database_table_definitions, row):
    
    table_definitions_column_indexes = {}
    
    for column in range(len(database_table_definitions[row])):
        table_definitions_column_indexes.update({database_table_definitions[row][column]:column})
    
    return table_definitions_column_indexes

In [5]:
def add_primary_key_constraints(generated_sql_code, table, primary_keys_list):
    
    primary_keys_string = str(primary_keys_list)
    primary_keys_string = primary_keys_string.replace("'", '')

    generated_sql_code += f'CONSTRAINT {table}_pk PRIMARY KEY ({primary_keys_string[1:-1]})'
    
    return generated_sql_code

In [6]:
def add_foreign_key_constraints(generated_sql_code, foreign_keys_dictionary):
    
    referenced_foreign_key_table = 'Placeholder_Referenced_Table'
    
    for table in foreign_keys_dictionary:
        foreign_key_constraints_count = 0
        
        for foreign_key_column in foreign_keys_dictionary.get(table):
            foreign_key_constraints_count += 1
            
            generated_sql_code += (
                '\n\n' + 
                f'ALTER TABLE {table} ' +
                f'ADD CONSTRAINT {table}_fk{foreign_key_constraints_count} ' + 
                f'FOREIGN KEY ({foreign_key_column}) ' + 
                f'REFERENCES {referenced_foreign_key_table}({foreign_key_column});'
            )
            
    return generated_sql_code

In [7]:
def convert_to_sql_code(csv_file_name):
    
    table_definitions_column_indexes = {}
    
    current_table = ''
    
    current_primary_keys_list = []
    primary_keys_string = ''
    
    current_foreign_keys_list = []
    foreign_keys_dictionary = {}
    
    generated_sql_code = ''

    db_tbl_defs = (
        read_database_table_definitions_from_csv_file('vehicle_database_sql_table_definitions.csv'))
    
    for row in range(len(db_tbl_defs)):
        generated_sql_code_length = len(generated_sql_code)

        if(row == 0):
            table_definitions_column_indexes = determine_table_definitions_column_indexes(db_tbl_defs, row)

        else:  
            if(current_table != db_tbl_defs[row][table_definitions_column_indexes.get('Table_Name')]):

                if(current_table != ''):

                    if(len(current_primary_keys_list) != 0):
                        generated_sql_code = add_primary_key_constraints(generated_sql_code, 
                                                                         current_table, 
                                                                         current_primary_keys_list)

                    if(len(current_foreign_keys_list) != 0):
                        foreign_keys_dictionary.update({current_table:current_foreign_keys_list})

                    generated_sql_code += '\n);\n\n' 

                current_table = db_tbl_defs[row][table_definitions_column_indexes.get('Table_Name')]

                current_primary_keys_list = []
                current_foreign_keys_list = []
                primary_keys_string = ''

                generated_sql_code += f'CREATE TABLE IF NOT EXISTS {current_table}' + '(' + '\n    '

            if(current_table == db_tbl_defs[row][table_definitions_column_indexes.get('Table_Name')]):
                generated_sql_code += f'{db_tbl_defs[row][table_definitions_column_indexes.get("Column_Name")]} '
                generated_sql_code += f'{db_tbl_defs[row][table_definitions_column_indexes.get("Data_Type")]}'

                if(db_tbl_defs[row][table_definitions_column_indexes.get('Data_Type')] != 'DATE' and 
                   db_tbl_defs[row][table_definitions_column_indexes.get('Data_Type')] != 'BOOLEAN'):
                    generated_sql_code += f'({db_tbl_defs[row][table_definitions_column_indexes.get("Length")]})'

                if(db_tbl_defs[row][table_definitions_column_indexes.get('PK')] == 'Yes'):
                    current_primary_keys_list.append(db_tbl_defs[row][table_definitions_column_indexes.get("Column_Name")])

                if(db_tbl_defs[row][table_definitions_column_indexes.get('Nullability')] == 'No'):
                    generated_sql_code += ' NOT NULL'

                if(db_tbl_defs[row][table_definitions_column_indexes.get('FK')] == 'Yes'):
                    current_foreign_keys_list.append(db_tbl_defs[row][table_definitions_column_indexes.get("Column_Name")])

                generated_sql_code += ',\n    '

        if(row == len(db_tbl_defs) - 1):

            if(len(current_primary_keys_list) != 0):    
                primary_keys_string = str(current_primary_keys_list)
                primary_keys_string = primary_keys_string.replace("'", '')

                generated_sql_code += f'CONSTRAINT {current_table}_pk PRIMARY KEY ({primary_keys_string[1:-1]})'

            generated_sql_code += '\n);'

            if(len(current_foreign_keys_list) != 0):
                foreign_keys_dictionary.update({current_table:current_foreign_keys_list})

    generated_sql_code = add_foreign_key_constraints(generated_sql_code, foreign_keys_dictionary)

    return generated_sql_code

In [8]:
generated_sql_code = convert_to_sql_code('vehicle_database_sql_table_definitions.csv')

print(generated_sql_code)

CREATE TABLE IF NOT EXISTS Car(
    ID VARCHAR(10) NOT NULL,
    Body_Style VARCHAR(20),
    Make VARCHAR(20),
    Model VARCHAR(20),
    Price DECIMAL(9) NOT NULL,
    Trim_Level VARCHAR(20),
    Year DATE,
    CONSTRAINT Car_pk PRIMARY KEY (ID)
);

CREATE TABLE IF NOT EXISTS Boat(
    ID VARCHAR(10) NOT NULL,
    Flying_Boat BOOLEAN,
    Name VARCHAR(20) NOT NULL,
    CONSTRAINT Boat_pk PRIMARY KEY (ID)
);

CREATE TABLE IF NOT EXISTS Flying_Boat(
    Boat_ID VARCHAR(10) NOT NULL,
    Plane_ID VARCHAR(10) NOT NULL,
    Information VARCHAR(100),
    CONSTRAINT Flying_Boat_pk PRIMARY KEY (Boat_ID, Plane_ID)
);

CREATE TABLE IF NOT EXISTS Plane(
    ID VARCHAR(10) NOT NULL,
    Flying_Boat BOOLEAN,
    Name VARCHAR(20) NOT NULL,
    CONSTRAINT Plane_pk PRIMARY KEY (ID)
);

CREATE TABLE IF NOT EXISTS Vehicle(
    ID VARCHAR(10) NOT NULL,
    Kind VARCHAR(20) NOT NULL,
    Details VARCHAR(10),
    CONSTRAINT Vehicle_pk PRIMARY KEY (ID)
);

ALTER TABLE Car ADD CONSTRAINT Car_fk1 FOREIGN KEY